## LSTM Example

We want to train a **multivariate**, **N** feature LSTM

**0) Loading Libraries and Subroutines**

Standard libraries for plotting and numerical operations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Loading LSTM related keras libraries:

In [ ]:
from keras import optimizers
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

Calling a subroutine that puts data set in the correct shape for LSTM (see later)

In [ ]:
from prepare_data import prepare_data

<br>

**1) Generating a Simple Dataset**

The goal is to generate a data set that has two features ($X(t_1)$ and $X(t_2)$) and to predict the values of two dependent variables $Y(t_1)$ and $Y(t_2)$

In [ ]:
t_start = -50
t_end   = 10
incr    = 0.25

t       = np.arange(t_start, t_end, incr)
t       = t.reshape(len(t), 1)

X_t1    = np.sin(t) + 0.1*np.random.randn(len(t), 1)
X_t2    = 0.1*np.random.randn(len(t),1) + 10*np.cos(t)

Y_t1    = X_t1 * X_t2
Y_t2    = X_t1 / (abs(X_t2) + 0.1)
Data    = np.hstack((Y_t1, Y_t2, X_t1, X_t2))

In [ ]:
fig, axs = plt.subplot_mosaic([['$Y_{t1}$'], ['$Y_{t2}$'], ['$X_{t1}$'], ['$X_{t2}$']], layout = 'constrained')
for i, l in enumerate(axs.items()):
    lab = l[0]
    axs[lab].plot(t, Data[:,i])
    axs[lab].set_title(lab)
plt.show()

**2) Preprocessing the Data**

2a) Scaling:

As before, we need to scale the data first.

In [ ]:
Xall    = np.hstack((X_t1, X_t2))
Yall    = np.hstack((Y_t1, Y_t2))

scaler  = MinMaxScaler(feature_range = (0, 1))

Y_tnorm = scaler.fit_transform(Yall)
X_tnorm = scaler.fit_transform(Xall)

2b) Reshaping the Data

In contrast to the previous example in LSTMI, the dependent variable is not just a prediction of $X(t)$ into the future, now it is two time series, $Y(t_1)$ and $Y(t_2)$. 

In [ ]:
dt_past    = 20
dt_futu    = 8
n_features = X_tnorm.shape[1]

[X, _]     = prepare_data(X_tnorm, dt_past, dt_futu)
[_, Y]     = prepare_data(Y_tnorm, dt_past, dt_futu)

<br>

Let us check the shapes:

In [ ]:
print(Y_tnorm.shape)
print(X_tnorm.shape)
print(X.shape)
print(Y.shape)

Each time series has two features now.

<br>

2c) Splitting data into Training and Test dataset

In [ ]:
cut            = int(np.round(0.7*Y_tnorm.shape[0]))

TrainX, TrainY = X[:cut], Y[:cut]
TestX,   TestY = X[cut:], Y[cut:]

Just to make sure, that the shape is *len(X) x dt_past x n_features*

In [ ]:
print(TrainX.shape)
print(TrainY.shape)

<br>

**3) Creating the Model**

Let us run two models: one for predicting $Y(t_1)$ and one for predicting $Y(t_2)$.<br>

In [ ]:
n_neurons  = 400
batch_size = 128

model1 = Sequential()
model1.add(LSTM(n_neurons, input_shape = (dt_past, n_features), activation = 'tanh'))
model1.add(Dense(dt_futu))

opt = optimizers.Adam()
model1.compile(loss = 'mean_squared_error', optimizer = opt)

model1.summary()

In [ ]:
model2 = Sequential()
model2.add(LSTM(n_neurons, input_shape = (dt_past, n_features), activation = 'tanh'))
model2.add(Dense(dt_futu))

opt = optimizers.Adam()
model2.compile(loss = 'mean_squared_error', optimizer = opt)

model2.summary()

<br>

**4) Running the Fit**

In [ ]:
n_epochs = 800
out1 = model1.fit(TrainX, TrainY[:,:,0], epochs = n_epochs, batch_size = batch_size, validation_split = 0.2, verbose = 2, shuffle = False)

In [ ]:
out2 = model2.fit(TrainX, TrainY[:,:,1], epochs = n_epochs, batch_size = batch_size, validation_split = 0.2, verbose = 2, shuffle = False)

In [ ]:
#plotting #############################################################
plt.plot(out1.history['loss'])
plt.plot(out1.history['val_loss'])
plt.title('model loss of $Y(t_1)$')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.savefig('training loss.pdf')
plt.show()
#######################################################################
#plotting #############################################################
plt.plot(out2.history['loss'])
plt.plot(out2.history['val_loss'])
plt.title('model loss of $Y(t_2)$')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.savefig('training loss.pdf')
plt.show()
#######################################################################

<br>

**5) Evaluating the Fit**

In [ ]:
PredY = model1.predict(TestX)
back  = PredY.shape[0]

plt.plot(t, Y_tnorm[:,0], linewidth = 5)
plt.plot(t[-back:], PredY[:, dt_futu-1])
plt.legend(['actual data', 'prediction'])
plt.fill_between([t[-back,0], t[-1,0]], 0, 1, color = 'k', alpha = 0.1)
plt.plot([t[-back,0], t[-back,0]], [0, 1], 'k-', linewidth = 3)
plt.show()

In [ ]:
PredY = model2.predict(TestX)
back  = PredY.shape[0]

plt.plot(t, Y_tnorm[:,1], linewidth = 5)
plt.plot(t[-back:], PredY[:, dt_futu-1])
plt.legend(['actual data', 'prediction'])
plt.fill_between([t[-back,0], t[-1,0]], 0, 1, color = 'k', alpha = 0.1)
plt.plot([t[-back,0], t[-back,0]], [0, 1], 'k-', linewidth = 3)
plt.show()